# 2A.ml - Pipeline pour un réduction d'une forêt aléatoire - énoncé

Le modèle Lasso permet de sélectionner des variables, une forêt aléatoire produit une prédiction comme étant la moyenne d'arbres de régression. Cet aspect a été abordé dans le notebook [Reduction d'une forêt aléatoire](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx3/notebooks/td2a_tree_selection_correction.html). On cherche à automatiser le processus.

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [2]:
%matplotlib inline

## Datasets

Comme il faut toujours des données, on prend ce jeu [Boston](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html).

In [3]:
from sklearn.datasets import load_boston
data = load_boston()
X, y = data.data, data.target

In [20]:
import pandas as pd

df = pd.DataFrame(X, columns=data.feature_names)

In [21]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

## Forêt aléatoire suivi de Lasso

La méthode consiste à apprendre une forêt aléatoire puis à effectuer d'une régression sur chacun des estimateurs.

In [5]:
import numpy
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso

# Apprentissage d'une forêt aléatoire
clr = RandomForestRegressor()
clr.fit(X_train, y_train)

# Récupération de la prédiction de chaque arbre
X_train_2 = numpy.zeros((X_train.shape[0], len(clr.estimators_)))
estimators = numpy.array(clr.estimators_).ravel()
for i, est in enumerate(estimators):
    pred = est.predict(X_train)
    X_train_2[:, i] = pred

# Apprentissage d'une régression Lasso
lrs = Lasso(max_iter=10000)
lrs.fit(X_train_2, y_train)
lrs.coef_

C:\Users\m_pic\AppData\Roaming\Python\Python37\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


array([0.02430339, 0.10029095, 0.        , 0.09685282, 0.13745066,
       0.11030709, 0.07509993, 0.16361483, 0.17455319, 0.13090604])

Nous avons réussi à reproduire le processus dans son ensemble. Pas toujours simple de se souvenir de toutes les étapes, c'est pourquoi il est plus simple de compiler l'ensemble dans un [pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

## Exercice 1 : Premier pipeline

Peut-être trouverez-vous tout de suite un [pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) qui fonctionne. La partie difficile est la partie qui produit le vecteur des sorties de chaque arbre de régression. La première piste que j'ai explorée est un [FunctionTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html).

In [13]:
from sklearn.pipeline import Pipeline

pipe = Pipeline(steps=[
    ('rf', RandomForestRegressor()),
    ("une fonction qui n'existe pas encore", fct),
    ("lasso", Lasso()),
])

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators='warn',
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)' (type <class 'sklearn.ensemble.forest.RandomForestRegressor'>) doesn't

Dans un pipeline, on ne peut y mettre que des modèles prédictifs, classifieur, régresseur ou des transformeur (normalisseur). La fonction qui extrait les prédictions des arbres doit être emballés dans un *transformer*. C'est le rôle d'un [FunctionTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html).

In [7]:
from sklearn.preprocessing import FunctionTransformer

def random_forest_tree_prediction(rf, X):
    preds = numpy.zeros((X.shape[0], len(rf.estimators_)))
    estimators = numpy.array(rf.estimators_).ravel()
    for i, est in enumerate(estimators):
        pred = est.predict(X)
        preds[:, i] = pred
    return preds
    

random_forest_tree_prediction(clr, X)

array([[24. , 24. , 24. , ..., 24. , 24. , 24. ],
       [29.1, 22.9, 21.4, ..., 29.1, 22.6, 22.2],
       [34.7, 34.7, 34.7, ..., 34.7, 34.9, 34.7],
       ...,
       [33.2, 28.7, 23.9, ..., 23.9, 23.9, 23.9],
       [28.1, 26.6, 27.9, ..., 23.9, 23.9, 23.9],
       [11.9, 11.9, 11.9, ..., 11.9, 20.4, 11.9]])

In [9]:
fct = FunctionTransformer(lambda X, rf=clr: random_forest_tree_prediction(rf, X) )

fct

FunctionTransformer(accept_sparse=False, check_inverse=True,
                    func=<function <lambda> at 0x0000029B138D61E0>,
                    inv_kw_args=None, inverse_func=None, kw_args=None,
                    pass_y='deprecated', validate=None)

In [ ]:
fct.transform(X_train)

In [12]:
try:
    pipe = Pipeline(steps=[
        ('rf', RandomForestRegressor()),
        ("extract_rf_preds", fct),
        ("lasso", Lasso()),
    ])
except Exception as e:
    print("Error: \n{}".format(e))

Error: 
All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators='warn',
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)' (type <class 'sklearn.ensemble.forest.RandomForestRegressor'>) doesn't


## Exercice 2 : Second pipeline

La première idée de marche pas vraiment... On décide alors de déguiser la forêt aléatoire en un transformeur.

In [7]:
class RandomForestRegressorAsTransformer:
    
    def __init__(self, **kwargs):
        self.rf = RandomForestRegressor(**kwargs)
        
    def fit(self, X, y):
        # ...
        return self
        
    def transform(self, X):
        # ...
        # return les prédiction de chaque arbre
        pass

# Tout ça pour écrire ce qui suit...
trrf = RandomForestRegressorAsTransformer()
trrf.fit(X_train, y_train)
trrf.transform(X_train)

Il reste à écrire le pipeline correspondant à la séquence d'apprentissage décrit quelque part dans ce notebook.

In [8]:
from sklearn.pipeline import Pipeline

pipe = Pipeline(steps=[
    ('name', 'passthrough'),
    # ...
])

pipe.fit(X_train, y_train)

Pipeline(memory=None, steps=[('name', 'passthrough')], verbose=False)

## Exercice 3 : GridSearchCV

Comme l'ensemble des traitements sont maintenant dans un seul pipeline que *scikit-learn* considère comme un modèle comme les autres, on peut rechercher les meilleurs hyper-paramètres du modèle, comme le nombre d'arbres initial, le paramètre *alpha*, la profondeur des arbres... Tout ça avec la classe [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

Vous devriez tomber sur un message disant que la classe ``RandomForestRegressorAsTransformer`` a besoin de la méthode *set_params*... Un indice : ``def set_params(self, **params): self.rf.set_params(**params)``.

## Exercice 4 : nombre de coefficients non nuls

Il ne reste plus qu'à trouver le nombre de coefficients non nuls du meilleur modèle, donc le nombre d'arbres conservés par le modèle.